In [61]:
### Functions

# ML PIPELINE

# 1. DATA INGESTION (READING IN DATA)
## Function to get data from a csv file
def get_data(url, drop=[]):
    import pandas as pd
    
    df = pd.read_csv(url)
    if len(drop) > 0:
        for col in drop:
            df.drop(columns=[col], inplace=True)

    return df

# 2. DATA CLEANING
## Function to group data in 'Other' category if less than a certain percentage of total
def bin_groups(df, percent):
    import pandas as pd

    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            for group, count in df[col].value_counts().iteritems():
                if count / len(df) < percent:
                    df.loc[df[col] == group, col] = 'Other'
    return df

## drop columns that are missing a certain percentage of data
def drop_columns_missing_data(df, cutoff):
    for col in df:
        if df[col].isna().sum() / len(df) > cutoff:
            df.drop(columns=[col], inplace=True)
    return df


## Replace nulls with mean
def impute_mean(df):
    from sklearn.impute import SimpleImputer
    import pandas as pd
    import numpy as np

    # Dummy code first; categorical features not allowed
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)

    # Change the strategy to mean, median, or mode
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)

    return df


## Replace nulls with predicted values using KNN (KNN is similar to clustering, only use on numerical data)
def impute_KNN(df):
    from sklearn.impute import KNNImputer
    from sklearn.preprocessing import MinMaxScaler
    import pandas as pd

    # Dummy code first; categorical features not allowed
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)

    # Clustering is biased by unstandardized data; so MinMax scale it
    df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns = df.columns)

    imp = KNNImputer(n_neighbors=5, weights="uniform")
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)

    return df


# Replace nulls with predicted values using MLR
def impute_reg(df):
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    import pandas as pd

# Dummy code first; categorical features not allowed
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)

    # Scaling is unnecessary for regression-based imputation

    imp = IterativeImputer(max_iter=10, random_state=12345)
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)

    return df


# 3. Model Data

def fit_mlr(df, test_size, random_state, label):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    import pandas as pd

    X = df.drop(label,axis=1)
    y = df[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LinearRegression().fit(X_train, y_train)
    print(f'R-squared (mlr): \t{model.score(X_test, y_test)}')

    return model


## Cross validation is a way to evaluate the performance of a model. Similar to training and testing your data. Just a little better way to do it. Newer too
def fit_crossvalidate_mlr(df, k, label, repeat=True):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    import pandas as pd
    from numpy import mean, std
    X = df.drop(label,axis=1)
    y = df[label]

    # prepare the cross-validation procedure
    if repeat:
        # splits will be how to split the data and repeats is how many times it will run so a split of 5 and a repeat of 4 would have 20 total models
        cv = RepeatedKFold(n_splits=k, n_repeats=5, random_state=12345)
    else:
        cv = KFold(n_splits=k, random_state=12345, shuffle=True)


    # evaluate model
    scores = cross_val_score(LinearRegression(), X, y, scoring='r2', cv=cv, n_jobs=-1)

    # report performance
    print(f'R-squared scores: \n{scores}\n')
    print(f'Average R-squared:\t{mean(scores)}')

    return LinearRegression().fit(X, y)


# 4. Deploy Model

## Save model
### pickle and joblib are two different packages you can use to save and load your models
def dump_pickle(model, file_name):
    import pickle
    pickle.dump(model, open(file_name, "wb"))

def dump_joblib(model, file_name):
    import joblib
    joblib.dump(model, file_name)

# Load models
def load_pickle(file_name):
    import pickle
    model = pickle.load(open(file_name, "rb"))
    return model
    
def load_joblib(file_name):
    import joblib
    model = joblib.load(file_name)
    return model

# Downcasting for smaller dataset
# Let's load the first dataset. We can us C engine to load faster those big files and take pandas function memory_usage to see what we gain. A conversion from bytes to megabytes is needed. So a division by 1024 ** 2 is need.
def memory(df):
    if isinstance(df,pd.DataFrame):
        value = df.memory_usage(deep=True).sum() / 1024 ** 2
    else: # we assume if not a df it's a series
        value = df.memory_usage(deep=True) / 1024 ** 2
    return value, "{:03.2f} MB".format(value)
print("Done")

def fs_variance(df, label="", p=0.02):
  from sklearn.feature_selection import VarianceThreshold
  import pandas as pd

  if label != "":
    X = df.drop(columns=[label])
    
  sel = VarianceThreshold(threshold=(p * (1 - p)))
  sel.fit_transform(X)

  # Add the label back in after removing poor features
  return df[sel.get_feature_names_out()].join(df[label])

Done


In [58]:
import pandas as pd

df = pd.read_csv("/Users/milesleung/Downloads/Utah_Crash_Data_2020.csv")

# data cleaning
df['CRASH_ID'] = df['CRASH_ID'].astype('int')
df['WORK_ZONE_RELATED'] = df['WORK_ZONE_RELATED'].astype('bool')
df['CRASH_DATETIME'] = pd.to_datetime(df['CRASH_DATETIME'])

dfIntSelection = df.select_dtypes(include=['int', 'float', 'bool', 'datetime64[ns]'])
# Drop object
df = dfIntSelection.apply(pd.to_numeric,downcast='unsigned')
memInt, memIntTxt =  memory(df)
memIntDownCast, memIntDownCastTxt = memory(df)

print(memIntTxt)
print(memIntDownCastTxt)
print('Gain: ', memInt/memIntDownCast *100.0)

df = df.drop(columns=['CRASH_ID', 'CRASH_DATETIME'])
df.info(memory_usage='deep')

13.48 MB
13.48 MB
Gain:  100.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252498 entries, 0 to 252497
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   MILEPOINT                      252479 non-null  float64
 1   LAT_UTM_Y                      252487 non-null  float64
 2   LONG_UTM_X                     252487 non-null  float64
 3   CRASH_SEVERITY_ID              252498 non-null  uint8  
 4   WORK_ZONE_RELATED              252498 non-null  bool   
 5   PEDESTRIAN_INVOLVED            252498 non-null  bool   
 6   BICYCLIST_INVOLVED             252498 non-null  bool   
 7   MOTORCYCLE_INVOLVED            252498 non-null  bool   
 8   IMPROPER_RESTRAINT             252498 non-null  bool   
 9   UNRESTRAINED                   252498 non-null  bool   
 10  DUI                            252498 non-null  bool   
 11  INTERSECTION_RELATED           252498 non-null  bool   
 12 

In [50]:
# Pipeline Option 2: Mean-substituted missing values
# Ingest data
df_mean = df
print('Data loaded')

# Bin categorical group values that are too rare in the dataset
# Drop any column with more than 50 percent of its data missing.
df_mean = drop_columns_missing_data(df_mean, .5)
# Replace the missing value with the mean, median, or mode of the column. Only works with numeric data.
df_mean = impute_mean(df_mean)
print('Data cleaned')

model_mlr = fit_crossvalidate_mlr(df_mean, 10, "CRASH_SEVERITY_ID", True)
print('model trained')

dump_pickle(model_mlr, "model_pickle_mean_prediction.sav")

Data loaded
Data cleaned
R-squared scores: 
[0.19305632 0.18511712 0.19544339 0.17882976 0.18623404 0.1909579
 0.19291684 0.19548581 0.18233355 0.1772825  0.17624787 0.18670682
 0.18935961 0.1946511  0.18555851 0.19510045 0.19082771 0.18516184
 0.18737322 0.18733167 0.17931693 0.18559323 0.19747439 0.18906989
 0.18148074 0.19447947 0.19121506 0.18953269 0.18481357 0.18516478
 0.19797939 0.1878141  0.18129101 0.19293733 0.18903869 0.18885794
 0.17942117 0.18882912 0.18237397 0.18977888 0.19359606 0.18025075
 0.20101302 0.19941783 0.16694657 0.19080521 0.19120226 0.18920534
 0.18091643 0.18391184]

Average R-squared:	0.1877940742861871
model trained


In [39]:
# Input 22 features into X
model = load_pickle("model_pickle_mean_prediction.sav")
round(model.predict([[12.702,4502005.00,414272.00,0,1,0,1,1,0,1,0,1,1,0,1,1,1,1,1,1,1,0]])[0], 2)

4.88

In [52]:
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
  
# Import data and follow prior cleaning steps
df_mean = df
print('Data loaded')
df_mean = bin_groups(df_mean, percent=0.05)
df_mean = drop_columns_missing_data(df_mean, .5)
df_mean = impute_mean(df_mean)
print('Data cleaned')
  
# Select features only
X = df_mean.drop(columns=['CRASH_SEVERITY_ID'])
  
# View the total number of features before reducing
print(X.shape)
  
# Calculate variance based on desired p-value so that Variance = p(1 - p)
# The code below uses p = 0.8 as the cutoff value:
p = 0.02
sel = VarianceThreshold(threshold=(p * (1 - p)))
sel.fit_transform(X)
X_reduced = pd.DataFrame(sel.fit_transform(X))
  
# View the remaining number of features
print(X_reduced.shape)
  
# Notice that columns are dropped
X_reduced.columns

Data loaded
Data cleaned
(252498, 22)
(252498, 16)


RangeIndex(start=0, stop=16, step=1)

In [56]:
# To see which columns were retained:
sel.get_feature_names_out()
df_mean[sel.get_feature_names_out()].columns

Index(['MILEPOINT', 'LAT_UTM_Y', 'LONG_UTM_X', 'WORK_ZONE_RELATED',
       'UNRESTRAINED', 'DUI', 'INTERSECTION_RELATED', 'WILD_ANIMAL_RELATED',
       'OVERTURN_ROLLOVER', 'COMMERCIAL_MOTOR_VEH_INVOLVED',
       'TEENAGE_DRIVER_INVOLVED', 'OLDER_DRIVER_INVOLVED',
       'NIGHT_DARK_CONDITION', 'SINGLE_VEHICLE', 'DISTRACTED_DRIVING',
       'ROADWAY_DEPARTURE'],
      dtype='object')

In [66]:
# Pipeline choreography
# Data cleaning and preparation pipeline
df_mean = df
print('Data loaded')
df_mean = bin_groups(df_mean, percent=0.05)
df_mean = drop_columns_missing_data(df_mean, .5)
df_mean = impute_mean(df_mean)
print('Data cleaned')

# Feature selection and modeling pipeline
df_mean = fs_variance(df_mean, label="CRASH_SEVERITY_ID", p=.02)
model = fit_crossvalidate_mlr(df_mean, 10, "CRASH_SEVERITY_ID", repeat=True)

# Deployment pipeline
dump_pickle(model, 'saved_model_prediction.sav')

Data loaded
Data cleaned
R-squared scores: 
[0.08728431 0.08874424 0.08920897 0.08198084 0.08475134 0.08601796
 0.08821824 0.08992969 0.08406669 0.08374945 0.08006488 0.09254788
 0.08020909 0.08719143 0.08167512 0.08712117 0.09110135 0.085459
 0.08836454 0.09046554 0.08346216 0.07850712 0.09401876 0.08699498
 0.08618142 0.09017861 0.08769236 0.0905389  0.0825718  0.08375358
 0.09427021 0.08977227 0.08269507 0.08684079 0.08515024 0.08521971
 0.08256666 0.08946789 0.08274005 0.08542111 0.0927442  0.08154326
 0.09948789 0.08649743 0.07869464 0.08358456 0.0825944  0.08495549
 0.08756879 0.08539465]

Average R-squared:	0.08638521473192762


In [70]:
### Finding the best Regression Model

def fit_crossvalidate_reg(df, label, k=10, r=5, repeat=True):
    import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    from numpy import mean, std
    from sklearn import svm
    from sklearn import gaussian_process
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
    from xgboost import XGBRegressor
    from sklearn import neighbors

    ## Setting labels and features
    X = df.drop(columns=[label])
    y = df[label]

    if repeat:
        cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
    else:
        cv = KFold(n_splits=k, random_state=12345, shuffle=True)

    fit = {}    # Use this to store each of the fit metrics
    models = {} # Use this to store each of the models

    # Create the model objects
    model_ols = lm.LinearRegression()
    model_rr = lm.Ridge(alpha=0.5) # adjust this alpha parameter for better results (between 0 and 1)
    model_lr = lm.Lasso(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
    model_llr = lm.LassoLars(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
    model_br = lm.BayesianRidge()
    model_pr = lm.TweedieRegressor(power=1, link="log") # Power=1 means this is a Poisson
    model_gr = lm.TweedieRegressor(power=2, link="log") # Power=2 means this is a Gamma
    model_igr = lm.TweedieRegressor(power=3) # Power=3 means this is an inverse Gamma
    model_abr = se.AdaBoostRegressor(n_estimators=100, random_state=12345)
    model_gbr = se.GradientBoostingRegressor(random_state=12345)
    model_hgbr = se.HistGradientBoostingRegressor(random_state=12345)
    model_xgb = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

    # Fit a crss-validated R squared score and add it to the dict
    fit['OLS'] = mean(cross_val_score(model_ols, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['LARS'] = mean(cross_val_score(model_llr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Bayesian'] = mean(cross_val_score(model_br, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Poisson'] = mean(cross_val_score(model_pr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Gamma'] = mean(cross_val_score(model_gr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Inverse'] = mean(cross_val_score(model_igr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['AdaBoost DT'] = mean(cross_val_score(model_abr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['Grad. Boost'] = mean(cross_val_score(model_gbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['HG Boost'] = mean(cross_val_score(model_hgbr, X, y, scoring='r2', cv=cv, n_jobs=-1))
    fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='r2', cv=cv, n_jobs=-1))

    # Add the model to another dict; make sure the keys have the same names as the list above
    models['OLS'] = model_ols
    models['Ridge'] = model_rr
    models['Lasso'] = model_lr
    models['LARS'] = model_llr
    models['Bayesian'] = model_br
    models['Poisson'] = model_pr
    models['Gamma'] = model_gr
    models['Inverse'] = model_igr
    models['AdaBoost DT'] = model_abr
    models['Grad. Boost'] = model_gbr
    models['HG Boost'] = model_hgbr
    models['XGBoost'] = model_xgb


    # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
    df_fit = pd.DataFrame({'R-squared':fit})
    df_fit.sort_values(by=['R-squared'], ascending=False, inplace=True)
    best_model = df_fit.index[0]
    print(df_fit)

    return models[best_model].fit(X, y)

# Data cleaning and preparation pipeline
df_mean = df
print('Data loaded')
df_mean = bin_groups(df_mean, percent=0.05)
df_mean = drop_columns_missing_data(df_mean, .5)
df_mean = impute_mean(df_mean)
print('Data cleaned')

# Feature selection and modeling pipeline
df_mean = fs_variance(df_mean, label="CRASH_SEVERITY_ID", p=.02)
model = fit_crossvalidate_reg(df_mean, 'CRASH_SEVERITY_ID', 5, 2)

# Deploy/store the model
dump_pickle(model, 'best_reg_model_prediction.sav')
print('Deploy/stored the model done')

Data loaded
Data cleaned
             R-squared
HG Boost      0.141068
XGBoost       0.130058
Grad. Boost   0.120411
Bayesian      0.086421
Ridge         0.086421
OLS           0.086421
Lasso         0.000550
Gamma        -0.000010
Inverse      -0.000010
Poisson      -0.000010
LARS         -0.000010
AdaBoost DT  -0.121998
Deploy/stored the model done


In [93]:
model = load_pickle('best_reg_model_prediction.sav')
print(pd.DataFrame({'Actual SalePrice':df_mean.CRASH_SEVERITY_ID, 'Predicted Crash Severity ID':model.predict(df_mean.drop(columns=['CRASH_SEVERITY_ID']))}).head())

   Actual SalePrice  Predicted Crash Severity ID
0               2.0                     2.070355
1               1.0                     1.372442
2               1.0                     1.268721
3               3.0                     2.550309
4               1.0                     1.286885


In [101]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 16]))]
onx = convert_sklearn(model, initial_types=initial_type)
with open("UtahCrash_HG_Boost_prediction.onnx", "wb") as f:
    f.write(onx.SerializeToString())